importing all the dependencies first

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

DATASET:
- 50k train images
- 10k test images
- 32*32 pixels RGB (meaning 3 channels)

In [2]:
# loading the dataset
(x_train,y_train), (x_test,y_test) = cifar10.load_data()
# normalize and convert type
x_train = x_train.astype("float32")/255.0
x_test = x_test.astype("float32")/255.0

- as we are using CNN, and we want spatial information to be preserved, we dont want to flatten our input.
- thus we can keep our input shape to (32,32,3)

#### Sequential Model

Building the model

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(32,32,3)),
        layers.Conv2D(32,3,padding='valid',activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64,3,activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128,3,activation='relu'),

        layers.Flatten(),
        layers.Dense(64,activation='relu'),
        layers.Dense(10)
    ]
)

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0

Compiling the model

In [5]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics = ['accuracy']
)

Training the model

In [6]:
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)

Epoch 1/10
782/782 - 19s - loss: 1.7061 - accuracy: 0.3763 - 19s/epoch - 25ms/step
Epoch 2/10
782/782 - 16s - loss: 1.3535 - accuracy: 0.5147 - 16s/epoch - 21ms/step
Epoch 3/10
782/782 - 14s - loss: 1.2178 - accuracy: 0.5683 - 14s/epoch - 17ms/step
Epoch 4/10
782/782 - 24s - loss: 1.1215 - accuracy: 0.6072 - 24s/epoch - 30ms/step
Epoch 5/10
782/782 - 22s - loss: 1.0492 - accuracy: 0.6331 - 22s/epoch - 29ms/step
Epoch 6/10
782/782 - 19s - loss: 0.9772 - accuracy: 0.6603 - 19s/epoch - 24ms/step
Epoch 7/10
782/782 - 15s - loss: 0.9289 - accuracy: 0.6771 - 15s/epoch - 19ms/step
Epoch 8/10
782/782 - 13s - loss: 0.8769 - accuracy: 0.6975 - 13s/epoch - 17ms/step
Epoch 9/10
782/782 - 17s - loss: 0.8363 - accuracy: 0.7107 - 17s/epoch - 22ms/step
Epoch 10/10
782/782 - 18s - loss: 0.8019 - accuracy: 0.7238 - 18s/epoch - 23ms/step


Model Evaluation:
- Returns the loss value & metrics values for the model in test mode.

In [7]:
model.evaluate(x_test,y_test,batch_size=64,verbose=2)

157/157 - 1s - loss: 0.8763 - accuracy: 0.7012 - 1s/epoch - 8ms/step


[0.8763088583946228, 0.701200008392334]

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0

#### Functional API

In [9]:
def my_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(32,3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64,5,padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(128,3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64,activation='relu')(x)
    outputs = layers.Dense(10)(x)

    model = keras.Model(inputs=inputs,outputs=outputs)
    return model

In [ ]:
model = my_model()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics = ['accuracy']
)
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)